<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AAI_CHROMA_GEMINI_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai crewai_tools langchain-google-genai chromadb pydantic -q
!pip install google-generativeai -q
!pip install langchain_community -q
!pip install crewai['tools'] -q
!pip install langchain-chroma -q

In [2]:
GOOGLE_API_KEY = None
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except (ImportError, KeyError):
    print("Not running in Google Colab or 'GEMINI' secret not found. Attempting to get 'GEMINI' environment variable.")
    GOOGLE_API_KEY = os.getenv('GEMINI')

Google Generative AI configured successfully using Colab Secrets.


In [ ]:
!pip install colab-env -q
import colab_env
!rm -rf /content/chroma_data

In [4]:
from litellm import completion
import warnings
warnings.filterwarnings("ignore")
import colab_env
import os

prompt="What is the capital of France?"

resp = completion(
    model="gemini/gemini-2.5-flash",
    messages=[{"role": "user", "content": prompt}],
    api_key=os.getenv('GEMINI'),
)

print(resp.choices[0].message.content)

The capital of France is Paris.


In [5]:
import os
import json
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import BaseTool
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import HumanMessage, AIMessage
from pydantic import BaseModel, Field
from typing import Type
import shutil
import time
import warnings
import datetime

# Suppress warnings
warnings.filterwarnings("ignore")

import google.generativeai as genai


# --- 1. Configuration for Agent ---
class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-2.5-flash"
    CREWAI_LLM_MODEL_NAME: str = "gemini/gemini-2.5-flash"


# --- 2. Google Colab / Gemini API Imports and Configuration ---
GOOGLE_API_KEY = None
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except (ImportError, KeyError):
    print("Not running in Google Colab or 'GEMINI' secret not found. Attempting to get 'GEMINI' environment variable.")
    GOOGLE_API_KEY = os.getenv('GEMINI')

if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print(f"Gemini API configured with model: {AgentConfig.LLM_MODEL_NAME}")
else:
    print("Warning: GOOGLE_API_KEY not found. LLM calls will not work.")
    print("Please set your 'GEMINI' environment variable or Colab secret.")

# --- 3. LLM Integration (CrewAI LLM for Gemini) ---
llm_for_crewai_agents = LLM(
    model=AgentConfig.CREWAI_LLM_MODEL_NAME,
    temperature=0.7,
    api_key=GOOGLE_API_KEY,
)


# --- 4. Vector DB Integration (ChromaDB for Memory) ---
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector_db = None

conversational_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# --- 5. Tools (Using BaseTool for Custom Tools) ---

class FlightSearchInput(BaseModel):
    origin: str = Field(description="The city or airport code of the departure location.")
    destination: str = Field(description="The city or airport code of the arrival location.")
    date: str = Field(description="The date of the flight in a clear format (e.g., 'July 20, 2025').")

class FlightSearchTool(BaseTool):
    name: str = "Flight Search Tool"
    description: str = "Useful for finding flight information between two cities on a specific date."
    args_schema: Type[BaseModel] = FlightSearchInput

    def _run(self, origin: str, destination: str, date: str) -> str:
        print(f"\n--- TOOL EXECUTION: Searching flights from {origin} to {destination} on {date} ---")
        if origin.lower() == "montreal" and destination.lower() == "toronto" and "july 20" in date.lower():
            return "Simulated Flight Results: AirCanada AC101 (9:00 AM - 10:15 AM) $150; WestJet WS205 (11:00 AM - 12:15 PM) $130. Please confirm to book."
        elif "paris" in origin.lower() and "london" in destination.lower():
            today_simulated = datetime.date(2025, 7, 2)
            tomorrow_simulated = today_simulated + datetime.timedelta(days=1)
            tomorrow_str = tomorrow_simulated.strftime("%B %d, %Y").lower()

            if "tomorrow" in date.lower() or tomorrow_str in date.lower():
                return "Simulated Flight Results: British Airways BA245 (10:30 AM - 11:30 AM) $220. Prices may vary."
            else:
                return "Simulated Flight Results: No direct flights found for your specified criteria. Consider alternative dates or nearby airports."
        else:
            return "Simulated Flight Results: No direct flights found for your specified criteria. Consider alternative dates or nearby airports."

    def _arun(self, origin: str, destination: str, date: str) -> str:
        raise NotImplementedError("Asynchronous _arun not implemented for FlightSearchTool.")

flight_tool_instance = FlightSearchTool()


# --- 6. CrewAI Agents ---
flight_planner = Agent(
    role='Expert Flight Planner',
    goal='Plan optimal flight itineraries based on user requests, considering preferences and real-time availability.',
    backstory=(
        "You are a highly efficient and knowledgeable AI flight planning assistant. "
        "You leverage powerful search tools and access to vast flight data to provide "
        "the best possible flight options. You also integrate memory for personalized service."
    ),
    verbose=False,
    allow_delegation=False,
    llm=llm_for_crewai_agents,
    tools=[flight_tool_instance]
)

memory_manager = Agent(
    role='Memory and Preference Manager',
    goal='Store and retrieve user preferences and past interactions for personalized flight planning.',
    backstory=(
        "You are the diligent keeper of user history and preferences. "
        "You seamlessly interact with the vector database to ensure that the flight planner "
        "always has access to relevant past information, making experiences personalized and efficient."
    ),
    verbose=False,
    allow_delegation=False,
    llm=llm_for_crewai_agents
)

# --- 7. CrewAI Tasks ---
search_flight_task = Task(
    description=(
        "Identify the origin, destination, and date from the user's request. "
        "Then, use the 'Flight Search Tool' to find available flights. "
        "Summarize the results clearly and ask the user for confirmation or further details."
        "Current user query: {query}"
    ),
    expected_output="A clear summary of flight options found (or not found) and a polite follow-up question.",
    agent=flight_planner,
)

store_preference_task = Task(
    description=(
        "Extract the user's preference from the provided 'preference_content'. "
        "Store this preference as a plain text string in the ChromaDB vector store "
        "using `vector_db.add_texts([preference_content])`. "
        "Confirm to the user that the preference has been stored."
        "Preference to store: {preference_content}"
    ),
    expected_output="A confirmation message that the user's preference has been successfully stored.",
    agent=memory_manager,
    human_input=False
)

retrieve_preference_task = Task(
    description=(
        "Based on the 'retrieve_query', search the ChromaDB vector store for relevant user preferences "
        "using `vector_db.similarity_search(retrieve_query, k=3)`. "
        "Summarize the retrieved preferences concisely for the user. If nothing is found, state that."
        "Query for retrieval: {retrieve_query}"
    ),
    expected_output="A summary of retrieved preferences or a statement indicating no relevant preferences were found.",
    agent=memory_manager,
    human_input=False
)

# --- Dynamic Interaction Loop (integrating conversational memory and vector DB) ---

def run_gemini_flight_agent(user_input: str) -> str:
    conversational_memory.chat_memory.add_user_message(user_input)

    chat_history_for_llm = "\n".join([f"{msg.type.capitalize()}: {msg.content}" for msg in conversational_memory.buffer_as_messages])

    if vector_db:
        vector_db.add_texts([f"User input: {user_input}"])
    else:
        print("Warning: vector_db is not initialized. Cannot add to long-term memory.")

    final_response = ""

    if "flight from" in user_input.lower() and "to" in user_input.lower():
        print("\n>>> Detected Flight Search Request. Initiating CrewAI Search Task...")
        inputs = {"query": user_input}
        flight_planning_crew = Crew(
            agents=[flight_planner],
            tasks=[search_flight_task],
            process=Process.sequential,
            verbose=False,
        )
        try:
            # Add a small delay before kickoff to mitigate rapid API calls
            time.sleep(1)
            crew_result = flight_planning_crew.kickoff(inputs=inputs)
            final_response = crew_result.raw
        except Exception as e:
            final_response = f"An error occurred during flight search: {type(e).__name__}: {e}. Please try again in a moment."
            print(f"Error during CrewAI kickoff: {e}")

    elif "remember that i prefer" in user_input.lower() or "store my preference" in user_input.lower():
        print("\n>>> Detected Preference Storage Request. Initiating CrewAI Store Preference Task...")
        preference_content = user_input.replace("remember that i prefer ", "").replace("store my preference ", "").strip()
        inputs = {"preference_content": preference_content}
        flight_planning_crew = Crew(
            agents=[memory_manager],
            tasks=[store_preference_task],
            process=Process.sequential,
            verbose=False,
        )
        try:
            time.sleep(1) # Small delay
            crew_result = flight_planning_crew.kickoff(inputs=inputs)
            final_response = crew_result.raw
        except Exception as e:
            final_response = f"An error occurred during preference storage: {type(e).__name__}: {e}. Please try again in a moment."
            print(f"Error during CrewAI kickoff: {e}")

    elif "what was my preference" in user_input.lower() or "retrieve my preference" in user_input.lower():
        print("\n>>> Detected Preference Retrieval Request. Initiating CrewAI Retrieve Preference Task...")
        inputs = {"retrieve_query": user_input}
        flight_planning_crew = Crew(
            agents=[memory_manager],
            tasks=[retrieve_preference_task],
            process=Process.sequential,
            verbose=False,
        )
        try:
            time.sleep(1) # Small delay
            crew_result = flight_planning_crew.kickoff(inputs=inputs)
            final_response = crew_result.raw
        except Exception as e:
            final_response = f"An error occurred during preference retrieval: {type(e).__name__}: {e}. Please try again in a moment."
            print(f"Error during CrewAI kickoff: {e}")

    else:
        print("\n>>> Detected General Query. Using LLM with combined memory (Direct Google Generative AI)..")
        relevant_docs = []
        if vector_db:
            relevant_docs = vector_db.similarity_search(user_input, k=3)
        else:
             print("Warning: vector_db is not initialized. Cannot retrieve from long-term memory.")

        context_from_db = "\n".join([doc.page_content for doc in relevant_docs])

        full_llm_prompt = (
            f"You are a helpful AI flight planning assistant. Engage in a natural conversation.\n"
            f"Here's the recent chat history:\n{chat_history_for_llm}\n\n"
            f"Here's relevant information from your long-term memory:\n{context_from_db}\n\n"
            f"User: {user_input}\n"
            f"Agent's Response:"
        )
        try:
            # Use google.generativeai.GenerativeModel directly for general queries
            model_direct = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
            time.sleep(1) # Small delay
            response_direct = model_direct.generate_content(full_llm_prompt)
            final_response = response_direct.text
        except Exception as e:
            final_response = f"An error occurred with the general LLM query (Direct GenAI): {type(e).__name__}: {e}. Please try again in a moment."
            print(f"Error during general LLM query (Direct GenAI): {e}")

    conversational_memory.chat_memory.add_ai_message(final_response)

    return final_response

if __name__ == "__main__":
    chroma_dir = "/content/chroma_data"
    if os.path.exists(chroma_dir):
        try:
            shutil.rmtree(chroma_dir)
            print(f"Cleared existing ChromaDB data directory: {chroma_dir}")
            time.sleep(1)
        except OSError as e:
            print(f"Error clearing chroma_data directory {chroma_dir}: {e}")

    try:
        vector_db = Chroma(
            collection_name="flight_planning_memory",
            embedding_function=embeddings_model,
            persist_directory=chroma_dir
        )
        print("ChromaDB initialized successfully.")
    except Exception as e:
        print(f"Error initializing ChromaDB: {e}")
        vector_db = None

    print(run_gemini_flight_agent("I need a flight from Montreal to Toronto on July 20, 2025."))
    print("\n" + "="*80 + "\n")

    print(run_gemini_flight_agent("Remember that I always prefer morning flights with window seats."))
    print("\n" + "="*80 + "\n")

    print(run_gemini_flight_agent("What was my seating preference?"))
    print("\n" + "="*80 + "\n")

    print(run_gemini_flight_agent("Tell me about the best time to visit Toronto."))
    print("\n" + "="*80 + "\n")

    print(run_gemini_flight_agent("I also need a flight from Paris to London tomorrow."))
    print("\n" + "="*80 + "\n")

    print("--- End of Agent Interaction ---")

Google Generative AI configured successfully using Colab Secrets.
Gemini API configured with model: gemini-2.5-flash
ChromaDB initialized successfully.

>>> Detected Flight Search Request. Initiating CrewAI Search Task...

--- TOOL EXECUTION: Searching flights from Montreal to Toronto on July 20, 2025 ---
I found two flight options from Montreal to Toronto on July 20, 2025:
*   AirCanada AC101 departing at 9:00 AM and arriving at 10:15 AM for $150.
*   WestJet WS205 departing at 11:00 AM and arriving at 12:15 PM for $130.

Would you like to proceed with booking one of these flights, or would you like to explore other options?



>>> Detected General Query. Using LLM with combined memory (Direct Google Generative AI)..
Okay, I've noted that you always prefer morning flights with window seats. I'll keep that in mind for future searches!

Regarding the flights for July 20, 2025, both AC101 (9:00 AM) and WS205 (11:00 AM) are morning flights. Which one would you like to book, or would you l